In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import importlib
import dataset
import json
import numpy as np
import os
import qa
import reasoning
import time
import torch
import utils


from dataset.bgquiz import BGQuiz
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForQuestionAnswering, BertForMultipleChoice
from qa.utils import query_es
from tqdm import tqdm_notebook
from utils.experiments import *
from reasoning.multichoice import *

# Reload the cached modules
_ = importlib.reload(dataset.bgquiz)
_ = importlib.reload(qa.utils)
_ = importlib.reload(reasoning.multichoice)
_ = importlib.reload(utils.experiments)

In [ ]:
DATA_PATH = "../data/bg_rc-v1.0.json"
MODEL_PATH = "../models/RACE_BERT/"
EXPERIMENTS_PATH = "../experiments"
# If the model is ran on GPUs
USE_CUDA = True


# Build the grid search for the experiment
grid = (GridSearch(MODEL_PATH)
        .set_indecies([
            'bgwiki_window', 'bgwiki_paragraph'
        ]).set_fields([
            ['title.bulgarian', 'passage.bulgarian'],
            ['title.bulgarian', 'passage.ngram'],
            ['title.bulgarian', 'passage.ngram', 'passage', 'passage.bulgarian'],
            ['passage.ngram', 'passage', 'passage.bulgarian^2'],
            ['title.bulgarian^2', 'passage.ngram', 'passage', 'passage.bulgarian^2'],
        ]).set_query_sizes([1, 2, 5, 10, 20]))


# Load the dataset
quiz = BGQuiz(DATA_PATH)

# Load the BERT number of choices should remain 4 for the bg_rc dataset
model = BertForMultipleChoice.from_pretrained(grid.model, num_choices=4)

# do_lower_case should be False for multicased model.
tokenizer = BertTokenizer.from_pretrained(grid.model, do_lower_case=False)  # Add specific options if needed
if USE_CUDA:
    _ = model.to('cuda')

In [ ]:
wiki_control = \
'''
Обсадата на Одрин приключва с превземането на турската крепост от Втора българска армия под общото командване на ген. Никола Иванов. Решителният пробив на източния сектор на крепостта е извършен под командването на ген. Георги Вазов. Ръководещият защитата на Одрин – Шукри паша, предава сабята си на ген. Иванов с думите: „Храбростта на българската армия е безподобна. На такава храброст никоя крепост не може да устои“. Победата фактически решава изхода на Балканската война.
'''

question_control = "Кои военачалници ръководят защитата и превземането на Одринската крепост през месец март 1913 г.?"
options_control = [
         "Абдулах паша – ген. Радко Димитриев",
         "Явер паша – ген. Васил Кутинчев",
         "Назим паша – ген. Данаил Николаев",
         "Шукри паша – ген. Никола Иванов"
      ]

question_control = \
'''
Името на коя международна организация е пропуснато в текста от нейния Устав, 
подписан през 1945 г. в Сан Франциско?    
„1. _____________________________ е основана върху принципа на пълното равенство на  своите членове. 
3. Те се задължават да уреждат международните си спорове с мирни  средства. 
4. Членовете се въздържат в отношенията си от заплаха или от употреба на  сила. 
7. Организацията не се намесва във вътрешните работи на държавите. 
Това обаче не  засяга прилагането на принудителни мерки в случай на заплаха срещу мира, 
на нарушение на  мира и на актове на агресия.”'''
wiki_control = '''
Никола Иванов (генерал)\n\nНикола Иванов Иванов е български офицер (генерал от пехотата), началник на Щаба на войската през 1894–1896 година и министър на войната (1896–1899). Командир на Втора българска армия през Балканските войни от 1912–1913 година. Ръководи обсадата и превземането на Одринската крепост (март 1913).\n\nНикола Иванов е роден на 2 март 1861 г. в Калофер. Учи в Априловската гимназия '''
options_control = [
         "Лигата на нациите",
         "Организацията на обединените нации",
         "Международната агенция за атомна енергия",
         "Международната организация на труда"
      ]
        
predict(model, tokenizer, wiki_control, question_control, options_control, use_gpu=USE_CUDA)

In [ ]:
def get_passages(es, setup, question, answers):
    passages = set()
    for a in answers:
        query = ' '.join((question, a))
        hits = query_es(es, query, setup.index, setup.query_size, setup.fields)
        passages.update((x['source']['passage'] for x in hits))

    return passages

In [ ]:
es = Elasticsearch()

for setup in grid():
    preds = {}
    print(setup)

    start = time.time()
    for i, q in tqdm_notebook(enumerate(quiz.iterator()), total=quiz.size()):
        question = q.question
        answers = q.answers

        try:
            # Collect passages
            passages = get_passages(es, setup, question, answers)

            # If number of answers is three, then add an empty option.
            ans_cnt = len(answers)
            if ans_cnt == 3:
                answers.append(' ')

            sc = np.zeros((1, 4), dtype=np.float32)
            # Calculate the evidence probability for each answer based on the retrieved passages.
            for p in passages:
                scores = predict(
                    model, tokenizer, p, question, answers, use_gpu=True)

                # Ignore the votes from empty options.
                if ans_cnt == 3:
                    scores[0, 3] = -np.inf
                sc += softmax(scores)

            preds[q.id] = answers[sc.argmax()]

        except KeyboardInterrupt:
            raise

    print()

    # Exports setup.json and pred.json
    dirname = str(int(datetime.now().timestamp()))
    path = os.path.join(EXPERIMENTS_PATH, dirname)
    export_predictions(path, setup, preds)

    # Evaluates the experiment using the default scripts
    print(path)
    !python evaluation/evaluate.py data/bg_rc-v1.0.json $path/preds.json  --out-file $path/eval.json
    !cat $path/eval.json
    print()

    end = time.time()
    print("Done in {} seconds".format(int(end - start)))
    print(''.center(40, '*'))
    print()